# Métrica 1.3.01: "Congruencia entre niveles de escolaridad finalizados"

# Métrica 1.3.02: "Congruencia en áreas de estudios profesionales finalizados"

Se han combinado las 2 metricas en un script por la cercania que tienen; este se ejecuta ingresando un nombre y no de forma iterativa como los demas.

In [14]:
import re
import urllib.parse
from pymongo import MongoClient, ASCENDING, DESCENDING
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
import traceback
from datetime import datetime
from collections import defaultdict # [NUEVO] Necesitamos esto para agrupar

# Importamos desde tu config
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_3_01_CONGRUENCIA_ESCOLARIDAD"
METRIC_ID_SECUENCIA = "1_3_02_SECUENCIA_TIPO" # [NUEVO] ID para tu nueva métrica

# --- 2. LÓGICA PURA DE LA MÉTRICA (Idéntica) ---

LEVEL_RANK_MAP = {
    "PRI": 1, "SEC": 2, "BCH": 3, "CTC": 3,
    "LIC": 4, "ESP": 5, "MTR": 5, "DOC": 6
}

def parse_iso_date(date_str: str) -> datetime:
    """
    Convierte un string de fecha ISO (YYYY-MM-DDTHH:MM:SSZ) a un objeto datetime.
    """
    if not isinstance(date_str, str): return None
    try:
        # Usamos [:-1] para quitar la 'Z' al final
        return datetime.strptime(date_str[:-1], "%Y-%m-%dT%H:%M:%S")
    except (ValueError, TypeError):
        return None

def get_max_escolaridad(escolaridad_array: list) -> int:
    """
    Recorre el arreglo de escolaridad y devuelve el RANGO MÁXIMO
    para los estudios que estén "FINALIZADO".
    Devuelve 0 si no se encuentra ninguno.
    """
    max_rank = 0
    if not isinstance(escolaridad_array, list): return 0
    for estudio in escolaridad_array:
        if isinstance(estudio, dict) and estudio.get("estatus") == "FINALIZADO":
            rank = LEVEL_RANK_MAP.get(estudio.get("nivel", {}).get("clave"))
            if rank and rank > max_rank:
                max_rank = rank
    return max_rank

def normalizar_tipo(tipo_str: str) -> str:
    """
    Convierte un string a mayúsculas, quita espacios y, lo más importante,
    elimina los acentos (ej. "MODIFICACIÓN" -> "MODIFICACION").
    """
    if not isinstance(tipo_str, str):
        return "DESCONOCIDO"
    
    # Convertir a mayúsculas y quitar espacios
    tipo_str = tipo_str.upper().strip()
    
    # Normalización robusta para quitar acentos
    # NFD = Normalization Form Decomposed (separa letra de acento)
    # encode/decode = quita los caracteres no-ASCII (los acentos)
    nfkd_form = unicodedata.normalize('NFD', tipo_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('ASCII')

# --- 3. EL FLUJO DEL SCRIPT INDIVIDUAL (CORREGIDO) ---
def procesar_historial_individual():
    """
    Pide un nombre completo, busca su historial, LO AGRUPA POR INSTITUCIÓN,
    y aplica la Métrica 1.3.01 (Escolaridad) y 1.3.02 (Secuencia)
    dentro de cada grupo.
    """
    client = None
    
    # --- A. OBTENER DATOS DEL USUARIO ---
    print("--- Análisis Individual de Métrica Longitudinal (1.3.01 y 1.3.02) ---")
    print("Por favor, introduce el nombre completo del declarante a procesar:")
    
    nombre = input("Nombre(s) (ej. ALEJANDRA): ").upper().strip()
    paterno = input("Primer Apellido (ej. HERNANDEZ): ").upper().strip()
    materno_input = input("Segundo Apellido (ej. PEREZ) (deja en blanco si no aplica): ").upper().strip()
    
    # Rutas de los campos
    path_nombre = "declaracion.situacionPatrimonial.datosGenerales.nombre" 
    path_paterno = "declaracion.situacionPatrimonial.datosGenerales.primerApellido"
    path_materno = "declaracion.situacionPatrimonial.datosGenerales.segundoApellido"
    path_fecha = "metadata.actualizacion"

    # --- B. CONSTRUIR FILTRO ROBUSTO ---
    filtro_busqueda = {
        path_nombre: nombre,
        path_paterno: paterno,
    }
    
    if materno_input == "":
        filtro_busqueda[path_materno] = { "$in": [None, ""] }
        print(f"\nBuscando historial para: {nombre} {paterno} (SIN APELLIDO MATERNO)")
    else:
        filtro_busqueda[path_materno] = materno_input
        print(f"\nBuscando historial para: {nombre} {paterno} {materno_input}")
    
    try:
        # --- C. CONECTAR Y ASEGURAR ÍNDICE ---
        print(f"Conectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        
        print("Asegurando el índice longitudinal...")
        # El índice sigue siendo útil para la búsqueda inicial
        source_collection.create_index([
            (path_nombre, ASCENDING),
            (path_paterno, ASCENDING),
            (path_materno, ASCENDING),
            (path_fecha, DESCENDING) 
        ])
        print(" 	> ¡Índice asegurado!")

        # --- D. BUSCAR HISTORIAL Y AGRUPAR POR INSTITUCION ---
        proyeccion = {
            "_id": 1, "id": 1, 
            "metadata.actualizacion": 1, 
            "metadata.institucion": 1,
            "metadata.tipo": 1, # [NUEVO] Necesitamos el tipo de declaración
            "declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad": 1
        }
        
        historial_cursor = source_collection.find(filtro_busqueda, proyeccion)
        
        # [MODIFICADO] Agrupamos los documentos por institución
        historial_por_institucion = defaultdict(list)
        
        for doc in historial_cursor:
            fecha_obj = parse_iso_date(doc.get("metadata", {}).get("actualizacion"))
            if fecha_obj:
                doc["fecha_obj"] = fecha_obj
                # [NUEVO] Usamos la institución como llave del diccionario
                institucion = doc.get("metadata", {}).get("institucion", "SIN_INSTITUCION_DEFINIDA")
                historial_por_institucion[institucion].append(doc)
            
        if not historial_por_institucion:
            print(f"--- NO SE ENCONTRARON REGISTROS ---")
            print(f"No se encontró ninguna declaración que coincida con esos criterios.")
            return

        print(f"Se encontraron declaraciones en {len(historial_por_institucion)} institución(es) distintas.")
        
        # [MODIFICADO] Ya no hay advertencia de homónimos, ahora es el flujo normal
        if len(historial_por_institucion) > 1:
            print("Se procesará el historial de cada institución por separado.")

        # --- E. PROCESAR EL HISTORIAL POR CADA INSTITUCIÓN ---
        resultados_guardados_totales = 0
        
        # [NUEVO] Bucle exterior: iteramos por cada institución encontrada
        for institucion, historial_institucion in historial_por_institucion.items():
            
            print(f"\n--- Procesando Institución: {institucion} ({len(historial_institucion)} registros) ---")
            
            # [MODIFICADO] Ordenamos por fecha solo la lista de esta institución
            historial_institucion.sort(key=lambda x: x["fecha_obj"])
            
            # [NUEVO] Estado inicial para la métrica de secuencia
            estado_secuencia = "ESPERA_INICIAL"
            
            # Bucle interior: iteramos por los documentos de esta institución
            for i in range(len(historial_institucion)):
                doc_actual = historial_institucion[i]
                id_actual = doc_actual["_id"]
                string_id = doc_actual.get("id", "N/D")
                tipo_bruto = doc_actual.get("metadata", {}).get("tipo")
                tipo_actual = normalizar_tipo(tipo_bruto)
                
                # Guardaremos ambas métricas
                metricas_a_guardar = {}
                
                # --- MÉTRICA 1: CONGRUENCIA ESCOLARIDAD (Tu lógica original) ---
                resultado_metrica_escolaridad = ""
                
                if i == 0:
                    resultado_metrica_escolaridad = "N/A" # Es la primera declaración EN ESTA INSTITUCIÓN
                else:
                    # [MODIFICADO] Comparamos con el doc anterior DE ESTA MISMA INSTITUCIÓN
                    doc_anterior = historial_institucion[i-1] 
                    
                    escolaridad_actual = doc_actual.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")
                    escolaridad_anterior = doc_anterior.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")

                    x_2 = get_max_escolaridad(escolaridad_actual)
                    x_1 = get_max_escolaridad(escolaridad_anterior)

                    if x_1 == 0 or x_2 == 0:
                        resultado_metrica_escolaridad = "SIN_DATO"
                    else:
                        diferencia = x_2 - x_1
                        if 0 <= diferencia <= 1:
                            resultado_metrica_escolaridad = "CUMPLE"
                        else:
                            resultado_metrica_escolaridad = "NO_CUMPLE"
                
                metricas_a_guardar[METRIC_ID] = resultado_metrica_escolaridad
                
                
                # --- MÉTRICA 2: SECUENCIA DE TIPO (Tu nueva lógica) ---
                resultado_metrica_secuencia = ""
                
                if estado_secuencia == "ESPERA_INICIAL":
                    if tipo_actual == "INICIAL":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                        estado_secuencia = "ESPERA_MODIFICACION_O_CONCLUSION" # Siguiente estado
                    else:
                        # Error: El primer doc (por fecha) no es INICIAL
                        resultado_metrica_secuencia = "ERROR_REVISION_INICIAL_FALTANTE"
                        estado_secuencia = "ERROR" # Se bloquea la secuencia para esta institución

                elif estado_secuencia == "ESPERA_MODIFICACION_O_CONCLUSION":
                    if tipo_actual == "MODIFICACION":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                        # Nos mantenemos en este estado, pueden venir más MODIFICACION
                    elif tipo_actual == "CONCLUSION":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                        estado_secuencia = "FINALIZADO" # Siguiente estado
                    elif tipo_actual == "INICIAL":
                        # Error: No puede haber otro INICIAL
                        resultado_metrica_secuencia = "ERROR_SECUENCIA_DOBLE_INICIAL"
                        estado_secuencia = "ERROR"
                    else:
                        resultado_metrica_secuencia = "ERROR_TIPO_DESCONOCIDO"
                        estado_secuencia = "ERROR"

                elif estado_secuencia == "FINALIZADO":
                    # Error: No debería haber nada después de una CONCLUSION
                    resultado_metrica_secuencia = "ERROR_SECUENCIA_POST_CONCLUSION"
                    estado_secuencia = "ERROR"

                elif estado_secuencia == "ERROR":
                    # La secuencia ya falló en un paso anterior
                    resultado_metrica_secuencia = "ERROR_SECUENCIA_PREVIA"

                else: # Por si acaso
                    resultado_metrica_secuencia = "ERROR_LOGICA_DESCONOCIDA"

                metricas_a_guardar[METRIC_ID_SECUENCIA] = resultado_metrica_secuencia

                
                # --- F. GUARDAR RESULTADOS (Ambas métricas) ---
                print(f"\n 	Procesando Documento ID: {id_actual} (id: {string_id})")
                print(f" 	> Fecha: {doc_actual['fecha_obj'].date()}, Tipo: {tipo_actual}")
                print(f" 	> Métrica Escolaridad: {resultado_metrica_escolaridad}")
                print(f" 	> Métrica Secuencia: {resultado_metrica_secuencia}")
                
                filtro = { "_id": id_actual }
                actualizacion = { "$set": metricas_a_guardar } # [MODIFICADO] Guarda ambas
                target_collection.update_one(filtro, actualizacion, upsert=True)
                print(f" 	> ¡Resultados guardados/actualizados en 'metricas'!")
                resultados_guardados_totales += 1

        print("\n--- PROCESAMIENTO INDIVIDUAL FINALIZADO ---")
        print(f"Se procesaron y guardaron {resultados_guardados_totales} resultados en total.")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        traceback.print_exc()
    finally:
        if client: client.close(); print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_historial_individual()

--- Análisis Individual de Métrica Longitudinal (1.3.01 y 1.3.02) ---
Por favor, introduce el nombre completo del declarante a procesar:

Buscando historial para: ALEJANDRA HERNANDEZ PEREZ
Conectando a MongoDB en sistema1...
¡Conexión exitosa!
Asegurando el índice longitudinal...
 	> ¡Índice asegurado!
Se encontraron declaraciones en 7 institución(es) distintas.
Se procesará el historial de cada institución por separado.

--- Procesando Institución: INSTITUTO DE EDUCACIÓN DE AGUASCALIENTES (GOBIERNO DEL ESTADO DE AGUASCALIENTES) (11 registros) ---

 	Procesando Documento ID: 68f81b8800535f910a2a0d33 (id: 190110)
 	> Fecha: 2024-05-30, Tipo: INICIAL
 	> Métrica Escolaridad: N/A
 	> Métrica Secuencia: SECUENCIA_CORRECTA
 	> ¡Resultados guardados/actualizados en 'metricas'!

 	Procesando Documento ID: 68f81b8800535f910a2a0d35 (id: 190148)
 	> Fecha: 2024-05-30, Tipo: MODIFICACION
 	> Métrica Escolaridad: CUMPLE
 	> Métrica Secuencia: SECUENCIA_CORRECTA
 	> ¡Resultados guardados/actualizad

In [15]:
import re
import urllib.parse
from pymongo import MongoClient, ASCENDING, DESCENDING
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
import traceback
from datetime import datetime
from collections import defaultdict 

# Importamos desde tu config
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
# [MODIFICADO] IDs de las métricas según tu nueva estructura
METRIC_ID_GRUPO_1_3_01 = "1_3_01"
METRIC_ID_CARRERAS_1_3_02 = "1_3_02"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

LEVEL_RANK_MAP = {
    "PRI": 1, "SEC": 2, "BCH": 3, "CTC": 3,
    "LIC": 4, "ESP": 5, "MTR": 5, "DOC": 6
}

def parse_iso_date(date_str: str) -> datetime:
    if not isinstance(date_str, str): return None
    try:
        return datetime.strptime(date_str[:-1], "%Y-%m-%dT%H:%M:%S")
    except (ValueError, TypeError):
        return None

# Lógica para 1.3.01 (Niveles)
def get_max_escolaridad(escolaridad_array: list) -> int:
    max_rank = 0
    if not isinstance(escolaridad_array, list): return 0
    for estudio in escolaridad_array:
        if isinstance(estudio, dict) and estudio.get("estatus") == "FINALIZADO":
            rank = LEVEL_RANK_MAP.get(estudio.get("nivel", {}).get("clave"))
            if rank and rank > max_rank:
                max_rank = rank
    return max_rank

# Funciones de ayuda para 1.3.02 (Carreras) y Secuencia
def normalizar_texto(texto_str: str) -> str:
    """ 
    Quita acentos, espacios y pasa a mayúsculas para comparar 
    carreras y tipos.
    """
    if not isinstance(texto_str, str): return ""
    
    texto_str = texto_str.upper().strip()
    
    nfkd_form = unicodedata.normalize('NFD', texto_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('ASCII')

def get_carreras_finalizadas(escolaridad_array: list, solo_licenciatura: bool = False) -> set:
    """
    Recorre el arreglo de escolaridad y devuelve un SET con los NOMBRES
    de las carreras/áreas de conocimiento que estén "FINALIZADO".
    """
    carreras = set()
    if not isinstance(escolaridad_array, list): return carreras
    
    for estudio in escolaridad_array:
        if isinstance(estudio, dict) and estudio.get("estatus") == "FINALIZADO":
            
            if solo_licenciatura:
                nivel_clave = estudio.get("nivel", {}).get("clave")
                if nivel_clave != "LIC":
                    continue 

            carrera = estudio.get("carreraAreaConocimiento")
            if carrera:
                carreras.add(normalizar_texto(carrera))
    return carreras
# --- Fin de lógicas puras ---

# --- 3. EL FLUJO DEL SCRIPT INDIVIDUAL (CORREGIDO) ---
def procesar_historial_individual():
    """
    Pide un nombre completo, busca su historial, LO AGRUPA POR INSTITUCIÓN,
    y aplica las Métricas 1.3.01 (Niveles y Secuencia) y 1.3.02 (Carreras)
    """
    client = None
    
    # --- A. OBTENER DATOS DEL USUARIO ---
    print(f"--- Análisis Individual de Métricas ({METRIC_ID_GRUPO_1_3_01} y {METRIC_ID_CARRERAS_1_3_02}) ---")
    print("Por favor, introduce el nombre completo del declarante a procesar:")
    
    nombre = input("Nombre(s) (ej. ALEJANDRA): ").upper().strip()
    paterno = input("Primer Apellido (ej. HERNANDEZ): ").upper().strip()
    materno_input = input("Segundo Apellido (ej. PEREZ) (deja en blanco si no aplica): ").upper().strip()
    
    path_nombre = "declaracion.situacionPatrimonial.datosGenerales.nombre" 
    path_paterno = "declaracion.situacionPatrimonial.datosGenerales.primerApellido"
    path_materno = "declaracion.situacionPatrimonial.datosGenerales.segundoApellido"
    path_fecha = "metadata.actualizacion"

    # --- B. CONSTRUIR FILTRO ROBUSTO ---
    filtro_busqueda = {
        path_nombre: nombre,
        path_paterno: paterno,
    }
    
    if materno_input == "":
        filtro_busqueda[path_materno] = { "$in": [None, ""] }
        print(f"\nBuscando historial para: {nombre} {paterno} (SIN APELLIDO MATERNO)")
    else:
        filtro_busqueda[path_materno] = materno_input
        print(f"\nBuscando historial para: {nombre} {paterno} {materno_input}")
    
    try:
        # --- C. CONECTAR Y ASEGURAR ÍNDICE ---
        print(f"Conectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        
        print("Asegurando el índice longitudinal...")
        source_collection.create_index([
            (path_nombre, ASCENDING),
            (path_paterno, ASCENDING),
            (path_materno, ASCENDING),
            (path_fecha, DESCENDING) 
        ])
        print(" 	> ¡Índice asegurado!")

        # --- D. BUSCAR HISTORIAL Y AGRUPAR POR INSTITUCION ---
        proyeccion = {
            "_id": 1, "id": 1, 
            "metadata.actualizacion": 1, 
            "metadata.institucion": 1,
            "metadata.tipo": 1, 
            "declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad": 1
        }
        
        historial_cursor = source_collection.find(filtro_busqueda, proyeccion)
        historial_por_institucion = defaultdict(list)
        
        for doc in historial_cursor:
            fecha_obj = parse_iso_date(doc.get("metadata", {}).get("actualizacion"))
            if fecha_obj:
                doc["fecha_obj"] = fecha_obj
                institucion = doc.get("metadata", {}).get("institucion", "SIN_INSTITUCION_DEFINIDA")
                historial_por_institucion[institucion].append(doc)
            
        if not historial_por_institucion:
            print(f"--- NO SE ENCONTRARON REGISTROS ---")
            print(f"No se encontró ninguna declaración que coincida con esos criterios.")
            return

        print(f"Se encontraron declaraciones en {len(historial_por_institucion)} institución(es) distintas.")
        
        if len(historial_por_institucion) > 1:
            print("Se procesará el historial de cada institución por separado.")

        # --- E. PROCESAR EL HISTORIAL POR CADA INSTITUCIÓN ---
        resultados_guardados_totales = 0
        
        for institucion, historial_institucion in historial_por_institucion.items():
            
            print(f"\n--- Procesando Institución: {institucion} ({len(historial_institucion)} registros) ---")
            
            historial_institucion.sort(key=lambda x: x["fecha_obj"])
            
            estado_secuencia = "ESPERA_INICIAL"
            
            for i in range(len(historial_institucion)):
                doc_actual = historial_institucion[i]
                id_actual = doc_actual["_id"]
                string_id = doc_actual.get("id", "N/D")
                tipo_bruto = doc_actual.get("metadata", {}).get("tipo")
                tipo_actual_norm = normalizar_texto(tipo_bruto) 
                
                # --- MÉTRICA 1.3.01: CONGRUENCIA NIVELES ---
                resultado_metrica_niveles = ""
                
                if i == 0:
                    resultado_metrica_niveles = "N/A" 
                else:
                    doc_anterior = historial_institucion[i-1] 
                    escolaridad_actual_arr = doc_actual.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")
                    escolaridad_anterior_arr = doc_anterior.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")

                    x_2 = get_max_escolaridad(escolaridad_actual_arr)
                    x_1 = get_max_escolaridad(escolaridad_anterior_arr)

                    if x_1 == 0 or x_2 == 0:
                        resultado_metrica_niveles = "SIN_DATO"
                    else:
                        diferencia = x_2 - x_1
                        if 0 <= diferencia <= 1:
                            resultado_metrica_niveles = "CUMPLE"
                        else:
                            resultado_metrica_niveles = "NO_CUMPLE"
                
                
                # --- MÉTRICA 1.3.02: CONGRUENCIA CARRERAS (LICENCIATURA) ---
                resultado_metrica_carreras = ""
                
                if i == 0:
                    resultado_metrica_carreras = "N/A"
                else:
                    doc_anterior = historial_institucion[i-1]
                    escolaridad_actual_arr = doc_actual.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")
                    escolaridad_anterior_arr = doc_anterior.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")

                    lic_actuales = get_carreras_finalizadas(escolaridad_actual_arr, solo_licenciatura=True)
                    
                    if not lic_actuales:
                        resultado_metrica_carreras = "N/A" 
                    else:
                        lic_anteriores = get_carreras_finalizadas(escolaridad_anterior_arr, solo_licenciatura=True)
                        
                        if not lic_anteriores:
                            resultado_metrica_carreras = "CUMPLE"
                        else:
                            if lic_anteriores.issubset(lic_actuales):
                                resultado_metrica_carreras = "CUMPLE"
                            else:
                                resultado_metrica_carreras = "NO_CUMPLE"

                
                # --- MÉTRICA: SECUENCIA DE TIPO (PARTE DE 1.3.01) ---
                resultado_metrica_secuencia = ""
                
                if estado_secuencia == "ESPERA_INICIAL":
                    if tipo_actual_norm == "INICIAL":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                        estado_secuencia = "ESPERA_MODIFICACION_O_CONCLUSION" 
                    else:
                        resultado_metrica_secuencia = "ERROR_REVISION_INICIAL_FALTANTE"
                        estado_secuencia = "ERROR" 

                elif estado_secuencia == "ESPERA_MODIFICACION_O_CONCLUSION":
                    if tipo_actual_norm == "MODIFICACION":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                    elif tipo_actual_norm == "CONCLUSION":
                        resultado_metrica_secuencia = "SECUENCIA_CORRECTA"
                        estado_secuencia = "FINALIZADO" 
                    elif tipo_actual_norm == "INICIAL":
                        resultado_metrica_secuencia = "ERROR_SECUENCIA_DOBLE_INICIAL"
                        estado_secuencia = "ERROR"
                    else:
                        resultado_metrica_secuencia = "ERROR_TIPO_DESCONOCIDO"
                        estado_secuencia = "ERROR"

                elif estado_secuencia == "FINALIZADO":
                    resultado_metrica_secuencia = "ERROR_SECUENCIA_POST_CONCLUSION"
                    estado_secuencia = "ERROR"

                elif estado_secuencia == "ERROR":
                    resultado_metrica_secuencia = "ERROR_SECUENCIA_PREVIA"

                else:
                    resultado_metrica_secuencia = "ERROR_LOGICA_DESCONOCIDA"


                # --- F. GUARDAR RESULTADOS (Todas las métricas) ---
                
                # [MODIFICADO] Construimos el objeto anidado para 1.3.01
                objeto_metrica_1_3_01 = {
                    "METRICA": resultado_metrica_niveles,
                    "SECUENCIA_TIPO": resultado_metrica_secuencia
                }
                
                # [MODIFICADO] Construimos el $set final
                metricas_a_guardar = {
                    METRIC_ID_GRUPO_1_3_01: objeto_metrica_1_3_01,
                    METRIC_ID_CARRERAS_1_3_02: resultado_metrica_carreras
                }
                
                # [MODIFICADO] Actualizamos los prints
                print(f"\n 	Procesando Documento ID: {id_actual} (id: {string_id})")
                print(f" 	> Fecha: {doc_actual['fecha_obj'].date()}, Tipo: {tipo_bruto} (Normalizado: {tipo_actual_norm})")
                print(f" 	> Métrica {METRIC_ID_GRUPO_1_3_01}.METRICA: {resultado_metrica_niveles}")
                print(f" 	> Métrica {METRIC_ID_GRUPO_1_3_01}.SECUENCIA_TIPO: {resultado_metrica_secuencia}")
                print(f" 	> Métrica {METRIC_ID_CARRERAS_1_3_02}: {resultado_metrica_carreras}")
                
                filtro = { "_id": id_actual }
                actualizacion = { "$set": metricas_a_guardar } 
                target_collection.update_one(filtro, actualizacion, upsert=True)
                print(f" 	> ¡Resultados guardados/actualizados en 'metricas'!")
                resultados_guardados_totales += 1

        print("\n--- PROCESAMIENTO INDIVIDUAL FINALIZADO ---")
        print(f"Se procesaron y guardaron {resultados_guardados_totales} resultados para estas métricas.")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        traceback.print_exc()
    finally:
        if client: client.close(); print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_historial_individual()

--- Análisis Individual de Métricas (1_3_01 y 1_3_02) ---
Por favor, introduce el nombre completo del declarante a procesar:

Buscando historial para: ALEJANDRA HERNANDEZ PEREZ
Conectando a MongoDB en sistema1...
¡Conexión exitosa!
Asegurando el índice longitudinal...
 	> ¡Índice asegurado!
Se encontraron declaraciones en 7 institución(es) distintas.
Se procesará el historial de cada institución por separado.

--- Procesando Institución: INSTITUTO DE EDUCACIÓN DE AGUASCALIENTES (GOBIERNO DEL ESTADO DE AGUASCALIENTES) (11 registros) ---

 	Procesando Documento ID: 68f81b8800535f910a2a0d33 (id: 190110)
 	> Fecha: 2024-05-30, Tipo: INICIAL (Normalizado: INICIAL)
 	> Métrica 1_3_01.METRICA: N/A
 	> Métrica 1_3_01.SECUENCIA_TIPO: SECUENCIA_CORRECTA
 	> Métrica 1_3_02: N/A
 	> ¡Resultados guardados/actualizados en 'metricas'!

 	Procesando Documento ID: 68f81b8800535f910a2a0d35 (id: 190148)
 	> Fecha: 2024-05-30, Tipo: MODIFICACIÓN (Normalizado: MODIFICACION)
 	> Métrica 1_3_01.METRICA: CUMP